# Data Preprocessing

### Importing Essential Libraries

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

### Importing csv data
- pandas.read_csv was used to open Data.csv(in same directory) as a DataFrame
- then we divided the dataset into independent variables(X) and dependent variables(y) using iloc that separates the dataset using indexes (before the comma are line indexes and after column ones)
- and finally transforming them into numpy arrays with ".values"

In [2]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
print('--------------dataset--------------')
print()
print(dataset)
print()
print('-----------------------------------')
print()
print(type(dataset))

--------------dataset--------------

   Country   Age   Salary Purchased
0   France  44.0  72000.0        No
1    Spain  27.0  48000.0       Yes
2  Germany  30.0  54000.0        No
3    Spain  38.0  61000.0        No
4  Germany  40.0      NaN       Yes
5   France  35.0  58000.0       Yes
6    Spain   NaN  52000.0        No
7   France  48.0  79000.0       Yes
8  Germany  50.0  83000.0        No
9   France  37.0  67000.0       Yes

-----------------------------------

<class 'pandas.core.frame.DataFrame'>


In [4]:
print('-------independent variables-------')
print()
pprint(X)
print()
print('-----------------------------------')
print()
print(type(X))

-------independent variables-------

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, nan],
       ['France', 35.0, 58000.0],
       ['Spain', nan, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

-----------------------------------

<class 'numpy.ndarray'>


In [5]:
print('--------dependent variables--------')
print()
pprint(y)
print()
print('-----------------------------------')
print()
print(type(y))

--------dependent variables--------

array(['No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes'],
      dtype=object)

-----------------------------------

<class 'numpy.ndarray'>


### Taking Care of Missing Data

In [6]:
from sklearn.impute import SimpleImputer

- first we have created an object from the SimpleImputer class, this objects receives which are the missing values that we want to replace (in this case NaN values) and the strategy we will use to replace the missing values (the mean between all other column values was chosen)
- then we use the fit function with the lines and columns we want to transform as parameters (in this case all lines of all numerical columns), this function will find the missing data and calculate the mean number
- last but not least we execute the imputer.transform function that will use the information acquired through the fit function to replace all missing data and we save the result in the right place of the X array

In [7]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 1:])
X[:, 1:] = imputer.transform(X[:, 1:])

In [8]:
pprint(X)

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, 63777.77777777778],
       ['France', 35.0, 58000.0],
       ['Spain', 38.77777777777778, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)


### Taking Care of Categorical Data

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#### Nominal Categorical Data
- at first we have created an object from the ColumnTransformer class
    - this object will receive 2 parameters, transformers an remainder, remainder will say what to do with the columns that we have not specified (in this case passthrough, that will keep the columns and do nothing with them), transformers is a little more complex, it will receive a list of tuples, each tuple will specify an action we want to take in a certain group of columns, the first element will be the transformer name('encoder'), the second one what we will use to transform(OneHotEncoder()) and the third element will be the columns in which we want to execute this action (we could use a list or a slice)
    - now talking more about the OneHotEncoder: It will create a column for each categorical value, and treat it somewhat like a boolean value. For exemple, we will have a column for Germany and in this column every line that had the "Germany" value will be filled in with 1 and every other line will be filled in with a 0

- then we will fit and transform it at once, make it a numpy array and then save again inside X

In [10]:
column_transformer = ColumnTransformer(transformers=[('encoder', 
                                                       OneHotEncoder(),
                                                       [0])],
                                       remainder='passthrough')
X = np.array(column_transformer.fit_transform(X))

In [11]:
pprint(X)

array([[1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0],
       [0.0, 1.0, 0.0, 30.0, 54000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0],
       [0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 35.0, 58000.0],
       [0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 0.0, 37.0, 67000.0]], dtype=object)


#### Boolean Categorical Data

- this time we will use LabelEncoder that is much simpler then ColumnTransformer and OneHotEncoder, and we can use it since we have only two values ('Yes' and 'No'). The LabelEncoder will transform y in a column with zeros and ones and each number will represent one categorical value (just like a boolean)

In [12]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [13]:
pprint(y)

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])


### Separating Training from Test set

In [14]:
from sklearn.model_selection import train_test_split

- for splitting the dataset into training and test set we will use the function train_test_split from the sklearn.model_selection library
- this function will receive X, y and the size of the test or training set as parameters. We could also use the parameter random_state if we wanted to obtain the same result everytime we execute the code, this parameter will be like a seed
- and it will return a list of arrays, in the order X_train, X_test, y_train, y_test
- we have to separate the sets to avoid overfitting, that would happen if we trained and tested in the same set (the model would be great to predict in this set but we would not be sure about other sets)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
pprint(X_train)

array([[1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 35.0, 58000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 1.0, 0.0, 30.0, 54000.0],
       [1.0, 0.0, 0.0, 37.0, 67000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0]], dtype=object)


In [17]:
pprint(X_test)

array([[0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0]], dtype=object)


In [18]:
pprint(y_train)

array([1, 1, 1, 0, 0, 0, 1, 1])


In [19]:
pprint(y_test)

array([0, 0])


### applying feature scaling in both Test and Training set
 - due to how feature scaling is done, we have to do feature scaling after separating the sets, otherwise some Training information would be inside the Test set and it cannot happen
 - we have two types of feature scaling, normalization and standardization. Normalization works when most of your features follow a normal distribution, while standardization works everytime, so we will use standardization this time
 - standardization will keep numbers between -3 and 3, while normalizarion keep numbers between 0 and 1
 - we don't need to use feature scaling in our dummy data (that we got from the categorical data) since it already is  0 or 1, and after using feature scalling we won't be abble to now which country is which
 

Normalization formula:
\begin{equation*}
x_{norm} = \frac{x - min(x)}{max(x) - min(x)}
\end{equation*}

Standardization formula:
\begin{equation*}
x_{stand} = \frac{x - mean(x)}{standard\_deviation(x)}
\end{equation*}

In [20]:
from sklearn.preprocessing import StandardScaler

- for the standardization we will use the StandardScaler class from the sklearn.preprocessing library
- first of all we simply created the object with no parameters, since we will use the default ones
- then we have used the fit_transform method in the part of the training set that we want to scale, this method will create the scaler with the mean and standard deviation and then apply it in the training set
- since we already have created the scaler, and we want to  use the same metrics in the training and test set, we will just use the transform method in the test set

In [21]:
standard_scaler = StandardScaler()
X_train[:, 3:] = standard_scaler.fit_transform(X_train[:, 3:])
X_test[:, 3:] = standard_scaler.transform(X_test[:, 3:])

In [22]:
print(X_train)

[[1.0 0.0 0.0 1.1868742003402273 1.1808268575196743]
 [0.0 1.0 0.0 0.14632695620632938 -0.16029877547676347]
 [1.0 0.0 0.0 -0.5040150713773568 -0.669339161723587]
 [0.0 1.0 0.0 1.4470110113737018 1.533239432613629]
 [1.0 0.0 0.0 0.6666005782732783 0.5641048511052539]
 [0.0 1.0 0.0 -1.154357098961043 -1.0217517368175415]
 [1.0 0.0 0.0 -0.24387826034388232 0.12358913223781073]
 [0.0 0.0 1.0 -1.5445623155112547 -1.5503705994584733]]


In [23]:
print(X_test)

[[0.0 0.0 1.0 -0.01264553942523824 -1.1979580243645187]
 [0.0 0.0 1.0 -0.11380985482714508 -0.40502973040312107]]
